In [11]:
# Essential Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting Style Configuration
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6) # Default size for figures

# =================================================================
# Data Loading
# =================================================================
# Professional Note: The file path points directly to the 'raw/' directory
# to maintain traceability and immutability of the original data.
try:
    df_train = pd.read_csv('data/raw/train.csv')
    df_test = pd.read_csv('data/raw/test.csv')
    print("Data successfully loaded.")
except FileNotFoundError:
    print("Error: Ensure that 'train.csv' and 'test.csv' files are in the 'data/raw/' folder.")

# Set 'Id' as the index for both DataFrames
df_train = df_train.set_index('Id')
df_test = df_test.set_index('Id')

Data successfully loaded.
